In [0]:
%run "/Users/bartoszgardzinski1@gmail.com/DE_Databricks_1stpipe/common_functions"

In [0]:
dbutils.widgets.text(name="env",defaultValue="",label="Enter the environment in lower case")
env = dbutils.widgets.get("env")

path = f"abfss://stock-project-container@bgardzinski1stock.dfs.core.windows.net/{env}/silver"

checkpoint_loc_fund = f"{path}/_checkpoints/prices_stream/"


fundamentals_int_columns = [
    "market_cap"]

non_null_columns = [
    "symbol",
    "open",
    "extract_time"]


def read_prices():
    return spark.readStream.table(f"{env}_bronze.prices")

def write_prices(df):
    df.writeStream.format("delta") \
    .option("checkpointLocation", checkpoint_loc_fund) \
    .trigger(availableNow=True) \
    .toTable(f"hive_metastore.{env}_silver.prices") \
    .awaitTermination()


df = read_prices()

#Cleaning and Type correction

df = filter_not_null(df, non_null_columns)
df = cast_ints(df, fundamentals_int_columns)
#watermark
df = df.withWatermark("extract_time", "2 days")
#drop duplicates
df = drop_duplicates_on_columns(df, non_null_columns)

write_prices(df)